# Step 2
distribuzione degli shots di ogni frammento tra più macchine

In [6]:

# definizione del circuito
edges = [
    (0, 1), (1, 2),
    (2, 3), (3, 4),
    (4, 5), (5, 6),
    (6, 7), (7, 8),
    (8, 9),
]
vertices = set()
for edge in edges:
    vertices.update(edge)

vertex_weights = {v: 1 for v in vertices}  # 1 qubit per gate in questo esempio

# definizione delle QPUs

class QPU: 
    def __init__(self, nome, tempo_di_esecuzione, tempo_di_coda, capacita, index):
        self.nome = nome
        self.tempo_di_esecuzione = tempo_di_esecuzione
        self.tempo_di_coda = tempo_di_coda
        self.capacita = capacita
        self.index = index

nomi = ['qpu_0', 'qpu_1', 'qpu_2', 'qpu_3', 'qpu_4']
tempo_di_esecuzione = [10, 200, 150, 300, 10] # di un singolo shot su una qpu
tempo_di_coda       = [ 1,  4,  3,  1,  3]
capacita            = [ 4,  4,  4,  4,  4]

qpus = []
for i in range(len(nomi)):
    qpus.append(QPU(nomi[i], tempo_di_esecuzione[i], tempo_di_coda[i], capacita[i], i))

# parametri per il CutQC dipendenti dalle QPU disponibili
num_subcircuits = len(qpus)  # Stessa dimensione dell'array QPU
num_qpus = len(qpus)
qpus_index = range(num_qpus) # indici dei qpu
subcircuits = range(num_subcircuits) # indici dei sottocircuiti


# numero di shots per sottocircuito
num_shots_per_subcircuit = 100 # uguale per tutti i sottosircuiti

## Definizione delle variabili principali

In [7]:
import pulp

problem = pulp.LpProblem("CircuitCutter_WithQPU", pulp.LpMinimize)

# y[v,c]: se il gate v appartiene al sottocircuito c
y = pulp.LpVariable.dicts("y",
    [(v, c) for v in vertices for c in subcircuits],
    cat=pulp.LpBinary)

# x[e,c]: se l'arco e è tagliato dal sottocircuito c
x = pulp.LpVariable.dicts("x",
    [(e, c) for e in edges for c in subcircuits],
    cat=pulp.LpBinary)

# a[c]: numero di qubit/gate originali inclusi in sottocircuito c
# p[c]: qubit di inizializzazione
# o[c]: qubit misurati in uscita
# f[c]: qubit che contribuiscono alla misura finale
# d[c]: numero totale di qubit in input al sottocircuito d[c] = a[c] + p[c]
a = pulp.LpVariable.dicts("a", subcircuits, cat=pulp.LpInteger)  # numero di gate nel sottocircuito
p = pulp.LpVariable.dicts("p", subcircuits, cat=pulp.LpInteger)  # qubit di init "aggiuntivi"
o = pulp.LpVariable.dicts("o", subcircuits, cat=pulp.LpInteger)  # qubit misurati in uscita
f = pulp.LpVariable.dicts("f", subcircuits, cat=pulp.LpInteger)  # qubit totali 'visti' = a[c] + p[c] - o[c]
d = pulp.LpVariable.dicts("d", subcircuits, cat=pulp.LpInteger)  # qubit in ingresso = a[c] + p[c]

# variabili per linearizzare i prodotti
# z_p[e, c]:  x[e,c] * y[e[1], c] 
# z_o[e, c]:  x[e,c] * y[e[0], c]
z_p = pulp.LpVariable.dicts(
    "z_p",
    [(e, c) for e in edges for c in subcircuits],
    cat=pulp.LpBinary
)
z_o = pulp.LpVariable.dicts(
    "z_o",
    [(e, c) for e in edges for c in subcircuits],
    cat=pulp.LpBinary
)

# variabili nuove per considerare anche gli shots

# shots_assign[c,q] = numero di shot del sottocircuito c assegnati alla QPU q
shots_assign = pulp.LpVariable.dicts(
    "shots_assign",
    [(c, q) for c in subcircuits for q in qpus_index],
    lowBound=0,
    upBound=num_shots_per_subcircuit,
    cat=pulp.LpInteger
)

# use_q[q] = 1 se la QPU q è utilizzata da almeno un sottocircuito
use_q = pulp.LpVariable.dicts(
    "use_q",
    qpus_index,
    cat=pulp.LpBinary
)

# T_q[q] = tempo totale di utilizzo della QPU q
T_q = pulp.LpVariable.dicts(
    "T_q",
    qpus_index,
    lowBound=0,
    cat=pulp.LpContinuous
)

# makespan (massimo tra i T_q)
T = pulp.LpVariable("Makespan", lowBound=0, cat=pulp.LpContinuous)


## Definizione dei vincoli

In [8]:
# vincoli su a[c], p[c], o[c], f[c], d[c]
for c in subcircuits:
    problem += a[c] == pulp.lpSum(vertex_weights[v]*y[v,c] for v in vertices), f"A_{c}"
    problem += p[c] == pulp.lpSum(z_p[(e,c)] for e in edges), f"P_{c}"
    problem += o[c] == pulp.lpSum(z_o[(e,c)] for e in edges), f"O_{c}"
    problem += f[c] == a[c] + p[c] - o[c], f"F_{c}"
    problem += d[c] == a[c] + p[c], f"D_{c}"

#linearizzazione per z_p ed z_o
for e in edges:
    for c in subcircuits:
        # z_p = x[e,c] * y[e[1], c]
        problem += z_p[(e, c)] <= x[(e, c)],           f"zp_1_{e}_{c}"
        problem += z_p[(e, c)] <= y[(e[1], c)],        f"zp_2_{e}_{c}"
        problem += z_p[(e, c)] >= x[(e, c)] + y[(e[1], c)] - 1, f"zp_3_{e}_{c}"

        # z_o = x[e,c] * y[e[0], c]
        problem += z_o[(e, c)] <= x[(e, c)],           f"zo_1_{e}_{c}"
        problem += z_o[(e, c)] <= y[(e[0], c)],        f"zo_2_{e}_{c}"
        problem += z_o[(e, c)] >= x[(e, c)] + y[(e[0], c)] - 1, f"zo_3_{e}_{c}"

# ogni vertice deve stare in esattamente un sottocircuito
for v in vertices:
    problem += pulp.lpSum(y[v,c] for c in subcircuits) == 1, f"Vertice_{v}_unico"

# vincoli sugli archi (no tagli se e[0] e e[1] sono nello stesso sottocircuito)
for c in subcircuits:
    for e in edges:
        problem += x[e,c] <= y[e[0],c] + y[e[1],c],         f"x_1_{e}_{c}"
        problem += x[e,c] >= y[e[0],c] - y[e[1],c],         f"x_2_{e}_{c}"
        problem += x[e,c] >= y[e[1],c] - y[e[0],c],         f"x_3_{e}_{c}"
        problem += x[e,c] <= 2 - y[e[0],c] - y[e[1],c],     f"x_4_{e}_{c}"

# vincolo per l'ordine
for k in range(num_subcircuits):
    problem += pulp.lpSum(y[(k, j)] for j in range(k+1, num_subcircuits)) == 0, f"Ordine_subc_{k}"


# Nuovi vincoli sugli shot e sulla capacità QPU

# gli shot di ogni sottocircuito devono sommare a num_shots_per_subcircuit
for c in subcircuits:
    problem += (
        pulp.lpSum(shots_assign[(c, q)] for q in qpus_index) == num_shots_per_subcircuit,
        f"ShotsTotali_sottoc_{c}"
    )

# attivazione QPU: se la QPU q ha shot > 0, allora use_q[q] = 1
M_shots = num_subcircuits * num_shots_per_subcircuit
for q in qpus_index:
    problem += (
        pulp.lpSum(shots_assign[(c, q)] for c in subcircuits) <= M_shots * use_q[q],
        f"UseQ_{q}_1"
    )

#capienza QPU: definiamo variabile ausiliaria "abilita[c,q]"
abilita = pulp.LpVariable.dicts(
    'abilita',
    [(c,q) for c in subcircuits for q in qpus_index],
    cat=pulp.LpBinary
)

# d[c] <= capacita[q] se abilita[c,q] = 1  altrimenti si neutralizza il vincolo.
# se abilita[c, q] = 0 => shots_assign[c,q] = 0 (non possiamo assegnare shot a una QPU non abilitata per quel c).
BigM_d = len(vertices) + len(edges)  # un big M per d[c]
for c in subcircuits:
    for q in qpus_index:
        # capacità
        problem += (
            d[c] <= qpus[q].capacita + BigM_d * (1 - abilita[(c,q)]),
            f"cap_{c}_{q}"
        )
        # se abilita[c, q] = 0 non si puo assegnare gli shot
        problem += (
            shots_assign[(c, q)] <= abilita[(c,q)] * num_shots_per_subcircuit,
            f"enable_shots_{c}_{q}"
        )


for q in qpus_index:
    # T_q >= tempo_coda_q * use_q[q]
    problem += (
        T_q[q] >= qpus[q].tempo_di_coda * use_q[q],
        f"TempoCodaMin_q{q}"
    )

    # T_q >= somma degli shots_assign * tempo_di_esecuzione
    problem += (
        T_q[q] >= pulp.lpSum(shots_assign[(c, q)] * qpus[q].tempo_di_esecuzione
                             for c in subcircuits),
        f"TempoEsecuzioneMin_q{q}"
    )

    # T_q <= tempo_coda_q + somma di esecuzione (se la QPU è usata)
    problem += (
        T_q[q] <= qpus[q].tempo_di_coda * use_q[q] +
                  pulp.lpSum(shots_assign[(c, q)] * qpus[q].tempo_di_esecuzione
                             for c in subcircuits),
        f"TempoMax_q{q}"
    )

    # makespan deve essere >= T_q[q] per ogni q
    problem += (T >= T_q[q], f"Makespan_{q}")


## Funzione obiettivo
K, ossia il numero di tagli, rimane invariata rispetto agli step precedenti mentre T, che rappresenta il tempo totale massimo necessario per completare l'esecuzione dei sottocircuiti distribuiti sulle QPU, viene calcolato tenendo conto del tempo totale di utilizzo di ciascuna QPU (T_q[q]) che include sia il tempo di coda che il tempo di esecuzione per gli shot assegnati. 

In [9]:
alpha = 0.6  # peso numero di tagli
beta  = 0.4  # peso makespan

if alpha < 0:
    raise Exception("Alpha deve essere maggiore o uguale a 0")
elif beta < 0:
    raise Exception("Beta deve essere maggiore o uguale a 0")
elif abs(alpha + beta) != 1.0:
    raise Exception("Alpha + Beta deve essere uguale di 1")

K = pulp.lpSum(x[e, c] for c in subcircuits for e in edges) / 2
K_max = len(edges) / 2  # massimo valore per K

# calcolo T_max come caso peggiore su QPU più lenta (coda + esecuzione su tutti i subcircuiti)
T_max = max(
    qpus[q].tempo_di_coda + num_subcircuits * num_shots_per_subcircuit * qpus[q].tempo_di_esecuzione
    for q in qpus_index
)

# normalizzazioni
K_norm = K / K_max
T_norm = T *(1/T_max)

# funzione obiettivo normalizzata
problem += alpha * K_norm + beta * T_norm, "Minimizza_Tagli_e_Makespan_Normalizzati"

problem.solve()

1

## Stampa risultati

In [10]:
print(f"Status: {pulp.LpStatus[problem.status]}")
print(f"Valore obiettivo: {pulp.value(problem.objective):.4f}\n")

for c in subcircuits:
    assegnazione = {}
    for q in qpus_index:
        val = pulp.value(shots_assign[(c, q)])
        if val > 0:
            assegnazione[qpus[q].nome] = int(val)
    print(f"Sottocircuito {c} -> Shots assegnati:", assegnazione)

print("\n-- Utilizzo QPU --")
for q in qpus_index:
    t_q_val = pulp.value(T_q[q])
    if t_q_val > 0:
        print(f"QPU {qpus[q].nome}: T_q = {t_q_val:.2f}, shots totali = "
              f"{sum(pulp.value(shots_assign[(c,q)]) for c in subcircuits)}")

print(f"\nMakespan T = {pulp.value(T):.2f}")


K_value = pulp.value(K)
print(f"Numero totale di cut = {K_value:.2f}")


Status: Optimal
Valore obiettivo: 0.2729

Sottocircuito 0 -> Shots assegnati: {'qpu_2': 15, 'qpu_3': 7, 'qpu_4': 78}
Sottocircuito 1 -> Shots assegnati: {'qpu_1': 11, 'qpu_4': 89}
Sottocircuito 2 -> Shots assegnati: {'qpu_0': 33, 'qpu_4': 67}
Sottocircuito 3 -> Shots assegnati: {'qpu_0': 100}
Sottocircuito 4 -> Shots assegnati: {'qpu_0': 100}

-- Utilizzo QPU --
QPU qpu_0: T_q = 2330.00, shots totali = 233.0
QPU qpu_1: T_q = 2200.00, shots totali = 11.0
QPU qpu_2: T_q = 2250.00, shots totali = 15.0
QPU qpu_3: T_q = 2100.00, shots totali = 7.0
QPU qpu_4: T_q = 2340.00, shots totali = 234.0

Makespan T = 2340.00
Numero totale di cut = 2.00
